In [27]:
import pandas as pd
import numpy as np
import seaborn as srn
import statistics as sts

In [3]:
# Importar dados
dataset = pd.read_csv("./dados/Churn.csv", sep=";")
# Visualizar
dataset.head()

,X0,X1,X2,X3,X4,X4.1,X6,X7,X8,X9,X10,X11
0,1,619,RS,Feminino,42,2,0,1,1,1,10134888.0,1
1,2,608,SC,Feminino,41,1,8380786,1,0,1,11254258.0,0
2,3,502,RS,Feminino,42,8,1596608,3,1,0,11393157.0,1
3,4,699,RS,Feminino,39,1,0,2,0,0,9382663.0,0
4,5,850,SC,Feminino,43,2,12551082,1,1,1,790841.0,0


In [4]:
# Tamanho
dataset.shape


(999, 12)

In [6]:

# Primeiro problema é dar nomes as colunas
dataset.columns = ["Id", "Score", "Estado", "Genero", "Idade", "Patrimonio",
                   "Saldo", "Produtos", "TemCartCredito", "Ativo", "Salario", "Saiu"]

In [ ]:
# Visualizar
dataset.head()

In [ ]:
# Explorar dados categóricos (Agrupar)
agrupado = dataset.groupby(['Estado']).size()
agrupado

In [ ]:
agrupado.plot.bar(color = 'gray')

In [ ]:
# Genero
agrupado = dataset.groupby(['Genero']).size()
agrupado

In [ ]:
agrupado.plot.bar(color = 'gray')

In [ ]:
# Explorar colunas numéricas
# Score
dataset['Score'].describe()

In [ ]:
srn.boxplot(dataset['Score']).set_title('Score')
# Histograma
srn.displot(dataset['Score'], kde = True)

In [ ]:
# Idade
dataset['Idade'].describe()

In [ ]:
srn.boxplot(dataset['Idade']).set_title('Idade')
srn.displot(dataset['Idade'], kde = True)

In [ ]:
# Saldo
dataset['Saldo'].describe()

In [ ]:
srn.boxplot(dataset['Saldo']).set_title('Saldo')

In [ ]:
srn.displot(dataset['Saldo'], kde = True)

In [7]:
# Salário
dataset['Salario'].describe()

count    9.920000e+02
mean     3.528762e+07
std      5.305800e+08
min      9.677000e+03
25%      3.029011e+06
50%      8.703250e+06
75%      1.405213e+07
max      1.193469e+10
Name: Salario, dtype: float64

In [ ]:
srn.boxplot(dataset['Salario']).set_title('Salario')

In [ ]:
srn.displot(dataset['Salario'], kde = True)

In [ ]:
# Contando valores NaN
# Genero e salário
dataset.isnull().sum()

In [ ]:
# Salários
dataset['Salario'].describe()

In [ ]:
mediana = sts.median(dataset['Salario'])
mediana

In [ ]:
dataset['Salario'].fillna(mediana, inplace = True)

In [8]:
# Verificando de NaN não existem mais
dataset['Salario'].isnull().sum()

7

In [ ]:
# Genero, falta de padronização e NaN's
agrupado = dataset.groupby(['Genero']).size()
agrupado

In [ ]:
# Total de NaN's 
dataset['Genero'].isnull().sum()

In [ ]:
# Preenche NaN's com Masculino (moda)
dataset['Genero'].fillna('Masculino', inplace = True)

In [ ]:
# Verificamos novamente NaN's
dataset['Genero'].isnull().sum()

In [ ]:
# Padronizando de acordo com o domínio
dataset.loc[dataset['Genero'] == 'M', 'Genero'] = 'Masculino'
dataset.loc[dataset['Genero'].isin(['Fem', 'F']), 'Genero'] = 'Feminino'

In [ ]:
# Visuaziando Resultado
agrupado = dataset.groupby(['Genero']).size()
agrupado

In [ ]:
# Idades fora do domínio
dataset['Idade'].describe()

In [ ]:
# Visualizando
dataset.loc[(dataset['Idade'] < 0) | (dataset['Idade'] > 120)]

In [ ]:
# Calculamos a mediana...
mediana = sts.median(dataset['Idade'])
mediana

In [ ]:
# Substituindo idades fora do domínio pela mediana
dataset.loc[(dataset['Idade'] < 0) | (dataset['Idade'] > 120), 'Idade'] = mediana

In [ ]:
# Verificando se ainda existem idades fora do domínio
dataset.loc[(dataset['Idade'] < 0) | (dataset['Idade'] > 120)]

In [ ]:
# Dados duplicados, buscamoe pelo ID
dataset[dataset.duplicated(['Id'], keep = False)]

In [ ]:
# Excluimos pelo ID
dataset.drop_duplicates(subset="Id", keep="first", inplace=True)
# Buscamos duplicados
dataset[dataset.duplicated(['Id'], keep = False)]

In [ ]:
# Estado fora do domínio
agrupado = dataset.groupby(['Estado']).size()
agrupado

In [ ]:
# Atribuímos RS (moda)
dataset.loc[dataset['Estado'].isin(['RP', 'SP', 'TD']), 'Estado'] = "RS"
agrupado = dataset.groupby(['Estado']).size()

In [ ]:
# Verificando o resultado
agrupado

In [30]:
# Outliers en salário, vamos considerar 2 desvios padrão
desv = np.std(dataset['Salario'])
desv

530312491.4410027

In [31]:
# Definindo padrão como maior que 2 desvios padrão
# Checando se algum atende o critério
dataset.loc[dataset['Salario'] >= 2 * desv]

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
7,8,376,PR,Feminino,29,4,11504674,4,1,0,1.193469e+10,1
116,118,668,PR,Feminino,37,6,1678644,1,1,0,1.156383e+10,0
170,172,484,RS,Feminino,29,4,13011439,1,1,0,1.640179e+09,0
230,232,673,RS,Masculino,72,1,0,2,0,1,1.119812e+09,0


In [32]:
# Atualizando salarios para a mediana, calculando:
mediana = sts.median(dataset['Salario'])
mediana

70518.0

In [33]:
# Atribuindo mediana
dataset.loc[dataset['Salario'] >= 2 * desv, 'Salario'] = mediana
# Checamos se algum atende o critério
dataset.loc[dataset['Salario'] >= 2 * desv]

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu


In [34]:
dataset.head()

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
0,1,619,RS,Feminino,42,2,0,1,1,1,10134888.0,1
1,2,608,SC,Feminino,41,1,8380786,1,0,1,11254258.0,0
2,3,502,RS,Feminino,42,8,1596608,3,1,0,11393157.0,1
3,4,699,RS,Feminino,39,1,0,2,0,0,9382663.0,0
4,5,850,SC,Feminino,43,2,12551082,1,1,1,790841.0,0


In [35]:
dataset.shape

(999, 12)